In [1]:
import requests
import pandas as pd
import urllib
import sys
import time

In [ ]:
country_code = {"ar": "Argentina", "at": "Austria", "au":"Australia", "br":"Brazil", "cl": "Chile", "de":"Germany", "es":"Spain", "fr":"France", "gb":"United Kingdom", "ge":"Georgia", "gr":"Greece", "it":"Italy", "pt":"Portugal", "us": "USA" }
type_code = {"1":"Red", "2":"White", "3":"Sparkling", "4": "Rose", "7": "Dessert", "24":"Fortified"}

In [ ]:
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()


    return data


In [ ]:
list_wines = []
price_max = "1000"
price_min = "96"
for i in range(1, 90):
    try:
        r = requests.get(
            "https://www.vivino.com/api/explore/explore",
            params={
                "country_code": "IT",
                "country_codes[]": "it",
                "currency_code": "EUR",
                "grape_filter": "varietal",
                "min_rating": "1",
                #"order_by": "ratings_average",
                "order_by": "price",
                "order": "asc",
                #"order": "asc",
                "page": i,
                "price_range_max": price_max,
                "price_range_min": price_min,
                "region_ids[]": 389, # piemonte
            },
            # headers={
            #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
            # },
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
            },
        )

        if not r.json()["explore_vintage"]:
            break

        results = [
            (
                t["vintage"]["wine"]["winery"]["name"],
                t["vintage"]["year"],
                t["vintage"]["wine"]["id"],
                f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
                t["vintage"]["statistics"]["ratings_average"],
                t["vintage"]["statistics"]["ratings_count"],
                type_code[str(t["vintage"]["wine"]["type_id"])],
                f'{t["vintage"]["wine"]["region"]["country"]["name"]} {t["vintage"]["wine"]["region"]["name"]}',
                t["vintage"]["wine"]["region"]["country"]["code"],
                f'{t["vintage"]["wine"]["region"]["country"]["most_used_grapes"][0]["name"]} {t["vintage"]["wine"]["region"]["country"]["most_used_grapes"][1]["name"]}',
                round(t["price"]["amount"], 2) if t["price"] is not None else "-",
                f'{"https://www.vivino.com/IT/en/"}{t["vintage"]["seo_name"]}{"/w/"}{t["vintage"]["wine"]["id"]}'
            )
            for t in r.json()["explore_vintage"]["matches"]
        ]
        dataframe = pd.DataFrame(
            results,
            columns=["Winery", "Year", "Wine ID", "Wine", "Rating", "num_review", "Wine type", "Wine region", "Country", "Grape", "price", "link"],
        )
    except Exception:
        print(f"Global exception occured on {i}")
        original_stdout = sys.stdout

        with open('error_page.txt', 'a') as f:
            sys.stdout = f
            print(f"Global exception occured on {i}")
            sys.stdout = original_stdout
        continue
    ratings = []
    for _, row in dataframe.iterrows():
        page = 1
        flag_for_old_links = 0
        while True:
            if flag_for_old_links == 0 and row["link"] in old_links:
                flag_for_old_links = 1
                break
            if page % 50 == 0 or page == 1: print(f'Getting info about wine {row["Wine"]} Page {page} Price {row["price"]}')
            iters = 0
            flag = 0
            while iters < 3:
                try:
                    d = get_wine_data(row["Wine ID"], row["Year"], page)
                    if not d["reviews"]:
                        flag = 1
                        break
                    for r in d["reviews"]:
                        #print(r["vintage"]["seo_name"])
                        if ("statistics" not in r["user"]) and ("vintage" not in r):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                "-",
                                "-",
                                "-",
                                r["language"],
                                "-"
                            ]
                        )
                            continue
                        if ("statistics" not in r["user"]):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                "-",
                                "-",
                                "-",
                                r["language"],
                                r["vintage"]["seo_name"]
                            ]
                        )
                            continue
                        if ("vintage" not in r):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                r["user"]["statistics"]["followers_count"],
                                r["user"]["statistics"]["followings_count"],
                                r["user"]["statistics"]["ratings_count"],
                                r["language"],
                                "-"
                            ]
                        )
                            continue


                        ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                r["user"]["statistics"]["followers_count"],
                                r["user"]["statistics"]["followings_count"],
                                r["user"]["statistics"]["ratings_count"],
                                r["language"],
                                r["vintage"]["seo_name"]
                            ]
                        )
                    break
                except Exception:
                    iters += 1
                    link = row["link"]
                    print(f"Exception occured on {page}, {link}, num of global page is {i}, num of iters {iters}")
                    original_stdout = sys.stdout  # Save a reference to the original standard output

                    with open('error_page.txt', 'a') as f:
                        sys.stdout = f  # Change the standard output to the file we created.
                        #print(dataframe["links"].drop_duplicates())
                        print(f"Exception occured on {page}, {link}, num of global page is {i}, num of iters {iters}")
                        sys.stdout = original_stdout
                        time.sleep(1)

            page += 1
            if flag == 1: break



    ratings = pd.DataFrame(
        ratings, columns=["Year", "Wine ID", "User's Grade", "Note", "CreatedAt", "user id", "followers", "following", "User Ratings", "language", "label"]
    )

    df_out = dataframe.merge(ratings)
    string = "wines_" + str(i) + ".csv"
    list_wines.append(df_out)
    # df_out.to_csv(string, index=False, sep = ';')
    print("Completed ",i)

string = "wines" + price_min +  "-" + price_max + ".csv"
result = pd.concat(list_wines)
result = result[["Winery", "Wine", "Rating", "num_review", "Wine type", "Wine region", "Country", "Grape", "price", "link", "Year", "Wine ID", "User's Grade", "CreatedAt", "user id", "followers", "following", "User Ratings", "language", "label"]]
result["image_name"]  = "images/" + result["Wine"].str.replace(" ", "") + ".png"
names_df = result[["Wine", "link", "image_name"]]
names_df = names_df.drop_duplicates()
result.to_csv(string, index=False, sep = ';')
names_df.to_csv(str("wine_names" + price_min + "-" + price_max + ".csv"), index=False, sep=';')

Getting info about wine Barolo Sorì Ginestra 2018-2018 Page 1 Price 96.98
Getting info about wine Fausoni Barbaresco 2013-2013 Page 1 Price 97.0
Getting info about wine Barbaresco Pajoré 2016-2016 Page 1 Price 97.0
Getting info about wine Arborina Barolo 2018-2018 Page 1 Price 97.0
Getting info about wine Barbaresco Nubiola 2011-2011 Page 1 Price 97.0
Getting info about wine Barolo Liste 2011-2011 Page 1 Price 97.0
Getting info about wine Barolo Vigna Rionda 2005-2005 Page 1 Price 97.0
Getting info about wine Barolo San Rocco 2013-2013 Page 1 Price 97.0
Getting info about wine Barolo Rocche di Castiglione 2015-2015 Page 1 Price 97.0
Getting info about wine Barolo Cannubi 2017-2017 Page 1 Price 97.6
Getting info about wine Barolo Monvigliero Riserva 2013-2013 Page 1 Price 97.9
Getting info about wine Vürsù Barbaresco Gallina 2014-2014 Page 1 Price 97.98
Getting info about wine Barolo Sorì Ginestra 2017-2017 Page 1 Price 97.98
Completed  1
Getting info about wine Mosconi Barolo 2019-2019

In [ ]:
df = pd.read_csv("wines25-96.csv", sep=";")
df.drop_duplicates()
df.to_csv("wines1_25-96.csv",sep=";", index=False)

<ipython-input-12-9010ac666c3a>:1: DtypeWarning: Columns (10,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("wines0-25.csv", sep=";")


In [ ]:
old_links = pd.read_csv("wine_names25-96.csv", sep = ";")
old_links = old_links["link"].to_list()



In [ ]:
df_1 = pd.read_csv("wine_names0-25.csv", sep=";")
df_2 = pd.read_csv("wine_names25-96.csv", sep=";")
df_3 = pd.read_csv("wine_names96-1000.csv", sep=";")
df_123 = pd.concat([df_1, df_2, df_3], ignore_index=True)
df_123 = df_123.drop_duplicates()
df_123.to_csv("wine_names_piemonte.csv",index=False, sep=';')

In [ ]:
df_123.head(10)

,Wine;link;image_name
0,White Sushi Wine N.V.;https://www.vivino.com/I...
1,Fragolino N.V.;https://www.vivino.com/IT/en/sa...
2,Il Sendallo Chardonnay 2022;https://www.vivino...
3,Buon Paggio Grignolino Piemonte 2022;https://w...
4,Contrada Maestra Barbera d'Asti 2022;https://w...
5,Barbera 2022;https://www.vivino.com/IT/en/noi-...
6,Dolcetto 2022;https://www.vivino.com/IT/en/noi...
7,Prima Mossa Barbera Piemonte 2022;https://www....
8,Capitano del Palio Dolcetto Monferrato 2022;ht...
9,Corsiero Biondo Cortese dell'Alto Monferrato 2...


In [4]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [27]:
df_for_images = pd.read_csv("wine_names_piemonte.csv", sep=';')
images = []
i = 0
wine_links = df_for_images['link']
for wine_link in wine_links:
  page = urlopen(wine_link)
  soup = BeautifulSoup(page)
  image = soup.findAll('img')[-1]["src"]
  images.append(image)
  print(i)
  print(df_for_images["Wine"][i])
  i+=1
df_for_images["image_link"] = images
df_for_images

Output hidden; open in https://colab.research.google.com to view.

In [8]:
pd.set_option('display.max_rows', 10000)

In [9]:
df_for_images

Output hidden; open in https://colab.research.google.com to view.

In [10]:
substring = "/images.vivino"
i = 0
for link in df_for_images["image_link"]:
  if substring not in link:
      print(i)
  i += 1



In [11]:
df_for_images["image_link"][1]

'//images.vivino.com/thumbs/fgUOE94-Tna3gznoOM8qOA_pb_x600.png'

In [30]:
for i in range(len(df_for_images["image_link"])):
  df_for_images["image_link"][i] = "https:" + df_for_images["image_link"][i]
df_for_images

Output hidden; open in https://colab.research.google.com to view.

In [24]:
image_names = []
for i in range(1513, len(df_for_images["image_link"])):
  url = df_for_images["image_link"][i]
  response = requests.get(url)
  image_name = str("images/"+ df_for_images.Wine[i].replace(" ","") + ".png")
  image_names.append(image_name)
  with open(str("/content/" + image_name), "wb") as f:
    f.write(response.content)
  print(i)
df_for_images["image_names"] = image_names
df_for_images

1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712


ValueError: ignored

In [29]:
df_for_images = df_for_images.rename(columns={"image_name": "image_names"})
df_for_images

Output hidden; open in https://colab.research.google.com to view.

In [17]:
df_for_images["Wine"][1512]

'Dolcetto d’Alba Bossèt (Boschetti/Buschet) 2021'

In [19]:
image_names[-1]

'images/Dolcettod’AlbaBossèt(Boschetti/Buschet)2021.png'

In [22]:
url = df_for_images["image_link"][1512]
response = requests.get(url)
with open("/content/images/Dolcettod’AlbaBossèt(Boschetti/Buschet)2021.png", "wb") as f:
  f.write(response.content)

FileNotFoundError: ignored

In [23]:
error_images = []
error_images.append(1512)

In [25]:
df_for_images

Output hidden; open in https://colab.research.google.com to view.

In [26]:
!zip -r /content/images.zip /content/images/

  adding: content/images/ (stored 0%)
  adding: content/images/TimorassoColliTortonesi2019.png (deflated 0%)
  adding: content/images/LaLusAlbarossa2019.png (stored 0%)
  adding: content/images/BaroloArione2018.png (deflated 0%)
  adding: content/images/BaroloCiabotMentin2012.png (stored 0%)
  adding: content/images/PoraBarbaresco2017.png (deflated 0%)
  adding: content/images/Barolo1942.png (deflated 0%)
  adding: content/images/BarbarescoAsili2010.png (stored 0%)
  adding: content/images/Barberad'Asti2020.png (deflated 2%)
  adding: content/images/Barolo1987.png (deflated 0%)
  adding: content/images/PoggiodelleAmareneBarberaColliTortonesi2019.png (stored 0%)
  adding: content/images/BaroloRavera2012.png (deflated 0%)
  adding: content/images/ConteisaLanghe2014.png (deflated 0%)
  adding: content/images/IlCastelloGavi2020.png (deflated 0%)
  adding: content/images/Rocchedell'AnnunziataTorriglioneBarolo2006.png (stored 0%)
  adding: content/images/UcelineMonferratoRosso2016.png (defla

In [31]:
df_for_images.to_csv("wine_names_piemonte.csv", sep=';', index=False)